In [1]:
import os

# Add OpenAI library
import openai

# Get Configuration Settings
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Configure OpenAI API using Azure OpenAI
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("ENDPOINT")
openai.api_type = "azure"  # Necessary for using the OpenAI library with Azure OpenAI
openai.api_version = "2024-02-01"  # Latest / target version of the API

In [10]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader

In [4]:
# OpenAI Settings
model_deployment = "text-embedding-ada-002"
# SDK calls this "engine", but naming it "deployment_name" for clarity

model_name = "text-embedding-ada-002"

In [5]:
openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_version = os.getenv("OPENAI_API_VERSION"), openai_api_key = os.getenv("API_KEY"),
    openai_api_base = os.getenv("ENDPOINT"), openai_api_type = "azure"
)

/Users/mac/Documents/Gospel-Companion/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Loading Junior and Senior Lesson Embeddings

In [6]:
from langchain_chroma import Chroma

In [7]:
# Junior
vector_store_junior = Chroma(
    collection_name="Junior_Lessons",
    embedding_function=openai_embeddings,
    persist_directory="/Users/mac/Documents/Gospel-Companion/chroma_afc_sunday_school_lessons_db",  # Where to save data locally, remove if not neccesary
)


In [8]:
# Senior
vector_store_senior = Chroma(
    collection_name="Senior_Lessons",
    embedding_function=openai_embeddings,
    persist_directory="/Users/mac/Documents/Gospel-Companion/chroma_afc_sunday_school_lessons_db",  # Where to save data locally, remove if not neccesary
)

### Prompt Structure

In [11]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory, ConversationBufferWindowMemory
from langchain import PromptTemplate

In [119]:
template = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

### Retrieval QA

In [120]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [121]:
import langchain
langchain.verbose = True

In [122]:
llm = ChatOpenAI(temperature = 0.4, openai_api_key = os.getenv("API_KEY"), openai_api_base = os.getenv("ENDPOINT"), model_name="gpt-35-turbo", engine="Voicetask")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


## Junior Lessons

In [123]:
retriever_j = vector_store_junior.as_retriever(search_kwargs={'k': 5})

### Similarity Search

In [124]:
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = retriever_j, 
    verbose = True,
    chain_type_kwargs = {
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
                k = 10,
                memory_key = "history",
                input_key = "question"),
                }
    )

In [125]:
query = "What are the steps to creation? Answer in bullet form using the days and the creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context given as "the Junior Sunday school lessons". 
------
<ctx>
1. How did the world come into existence?   How do we know?

2. What did God do on the first day?

3. What did God say at the close of every day of creation?

4. Why did God create the world?

5. Why does the ocean stay within its bounds?

6. What different kinds of flesh are there?

First, God spoke the heavens and the earth into existence, apparently a mass of substance, hidden in darkness.  Then He began to bring order out of that which was without for


> Finished chain.

> Finished chain.

> Finished chain.
1. God spoke the heavens and the earth into existence
2. On the first day, God made light
3. On the second day, God divided the waters above from the waters below to create the sky
4. On the third day, God created dry land and vegetation
5. On the fourth day, God created the sun, moon, and stars
6. On the fifth day, God created sea creatures and birds
7. On the sixth day, God created land animals and humans
8. On the seventh day, God rested.


In [126]:
query = "What is the proof that God actually rested on the 7th day of creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context given as "the Junior Sunday school lessons". 
------
<ctx>
While God, with the completion of the creation, rested on the seventh day and hallowed it, yet it was not given to the Israelites to observe this day until the Law was written, when the seventh day was established as a Sabbath in commemoration of their deliverance out of bondage ([DEU:5:15]).  Jesus emphasised the necessity of Christians observing in spirit all the other laws given in the Ten Commandments but He never repeated the command to observe the S


> Finished chain.

> Finished chain.

> Finished chain.
The Bible states that God rested on the seventh day of creation and hallowed it, but there is no physical proof of this event. It is a matter of faith and belief in the biblical account of creation. The Junior Sunday school lessons explain that the Sabbath was not given to the Israelites to observe until the Law was written, and Jesus never repeated the command to observe the Sabbath.


In [127]:
query = "What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context given as "the Junior Sunday school lessons". 
------
<ctx>
The Lord's Day

One more instruction given in this lesson is to keep the Sabbath Day holy. One of the Ten Commandments given to the Children of Israel many years before the time of Isaiah was: "Remember the Sabbath day, to keep it holy" [EXO:20:8]). The first day of the week became accepted as the day of worship immediately after Jesus' resurrection, and has been observed by all true Christians from that time down to the present day. It is the day on whic

In [128]:
query = "What are some of the most controversial and debated doctrines of the Bible?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context given as "the Junior Sunday school lessons". 
------
<ctx>
Christians are warned specifically against those who say we must keep the forms and ceremonies of the Law of Moses, whereas the Bible says that, "by the deeds of the law there shall no flesh be justified in his sight" [ROM:3:20]). We are warned against those who deny that Jesus is the Son of God who came to earth to die and rise again to save us from sin, whereas the Bible says, "Whosoever shall confess that Jesus is the Son of God, God dwelleth in him, a

In [129]:
query = "What about eternal Salvation, when one believes that once he is born again, he can continue in sin but he still remians born again."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context given as "the Junior Sunday school lessons". 
------
<ctx>
Because a person is saved at one time, does not necessarily mean that he will always be saved. Unless he lives for Jesus and does not permit sin to enter his life he will lose his right to eternal life [EZE:18:26]). There is power in the Blood of Jesus to save, and to keep, if a person wants to be kept in the Lord.

Resisting Temptation

There are people today who feel that if they go to church on Sunday morning, and never do anyone any harm, they will go

In [130]:
query = "What can you tell me about the book of Hebrews that a new Christian should know?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context given as "the Junior Sunday school lessons". 
------
<ctx>
Bible Text: [HEB:1:1-14].

Lesson 432 - Junior

Memory Verse: [HEB:1:1-14].

Notes"Go ye therefore, and teach all nations, baptizing them in the name of the Father, and of the Son, and of the Holy  Ghost" (Matthew 28:19).

The Sacrifices Fulfilled

The Book of Hebrews was written by the Apostle Paul to the Hebrew Christians, those who had believed on Jesus. They had been brought up to obey the Law of Moses, and they knew all about the sacrifices that shou

In [131]:
qa_stuff.combine_documents_chain.memory.load_memory_variables({})

{'history': 'Human: What are the steps to creation? Answer in bullet form using the days and the creation\nAI: 1. God spoke the heavens and the earth into existence\n2. On the first day, God made light\n3. On the second day, God divided the waters above from the waters below to create the sky\n4. On the third day, God created dry land and vegetation\n5. On the fourth day, God created the sun, moon, and stars\n6. On the fifth day, God created sea creatures and birds\n7. On the sixth day, God created land animals and humans\n8. On the seventh day, God rested.\nHuman: What is the proof that God actually rested on the 7th day of creation\nAI: The Bible states that God rested on the seventh day of creation and hallowed it, but there is no physical proof of this event. It is a matter of faith and belief in the biblical account of creation. The Junior Sunday school lessons explain that the Sabbath was not given to the Israelites to observe until the Law was written, and Jesus never repeated t

In [132]:
print(qa_stuff.combine_documents_chain.memory.buffer)

Human: What are the steps to creation? Answer in bullet form using the days and the creation
AI: 1. God spoke the heavens and the earth into existence
2. On the first day, God made light
3. On the second day, God divided the waters above from the waters below to create the sky
4. On the third day, God created dry land and vegetation
5. On the fourth day, God created the sun, moon, and stars
6. On the fifth day, God created sea creatures and birds
7. On the sixth day, God created land animals and humans
8. On the seventh day, God rested.
Human: What is the proof that God actually rested on the 7th day of creation
AI: The Bible states that God rested on the seventh day of creation and hallowed it, but there is no physical proof of this event. It is a matter of faith and belief in the biblical account of creation. The Junior Sunday school lessons explain that the Sabbath was not given to the Israelites to observe until the Law was written, and Jesus never repeated the command to observe t